# 1.Import data

In [ ]:
#!pip install snowflake-connector-python
#!pip install snowflake-connector-python[pandas]
#!pip install usaddress
import snowflake.connector
from snowflake.connector import cursor
import usaddress
import re

In [ ]:
ctx = snowflake.connector.connect(
    user='',
    password='',
    account=''
    )

cursor = ctx.cursor()

In [ ]:
import pandas as pd
sql_MLS = """"""
df_MLS = pd.read_sql(sql_MLS, con=ctx)

In [ ]:
sql_DATATREE = """"""
df_DATATREE = pd.read_sql(sql_DATATREE, con=ctx)

In [ ]:
df_MLS

In [ ]:
df_DATATREE

In [ ]:
len(df_MLS['MLS_ADDRESS'].unique())/len(df_DATATREE['DATATREE_ADDRESS'].unique())

# 2. First Node - Completely match

In [ ]:
# Drop NaN
df_MLS.dropna(subset=['MLS_ADDRESS'], inplace=True)
df_DATATREE.dropna(subset=['DATATREE_ADDRESS'], inplace=True)


In [ ]:
# Merge the data
df_merge_node1 = pd.merge(df_DATATREE,df_MLS, how = 'left', right_on = ['MLS_ADDRESS', 'POSTALCODE'], 
                          left_on = ['DATATREE_ADDRESS', 'SITUSZIP5'])


In [ ]:
# Compute ratio
first_node_match = len(df_merge_node1[df_merge_node1['MLS_ADDRESS'].notnull()]['MLS_ADDRESS'].unique())
base = len(df_DATATREE['DATATREE_ADDRESS'].unique())
first_node_match_ratio = first_node_match/base
first_node_match_ratio


- Create Matched addresses list

In [ ]:
first_node_match

In [ ]:
MLS_matched_address = []
DATATREE_matched_address = []

In [ ]:
first_node_match_list = list(df_merge_node1[df_merge_node1['MLS_ADDRESS'].notnull()]['MLS_ADDRESS'].unique())
MLS_matched_address.extend(first_node_match_list)
DATATREE_matched_address.extend(first_node_match_list)

In [ ]:
len(MLS_matched_address)

In [ ]:
len(DATATREE_matched_address)

# 3. Second Node - Match after transformation

- Remove punctuation

In [ ]:
# Remove punctuations
from string import punctuation
import re
# punctuation -> space
remove_punctiuation = str.maketrans(punctuation, ' '*len(punctuation))

df_MLS['MLS_ADDRESS'] = df_MLS['MLS_ADDRESS'].apply(lambda x: x.translate(remove_punctiuation))
df_DATATREE['DATATREE_ADDRESS'] = df_DATATREE['DATATREE_ADDRESS'].apply(lambda x: x.translate(remove_punctiuation))
# several spaces -> one space
df_MLS['MLS_ADDRESS'] = df_MLS['MLS_ADDRESS'].apply(lambda x: re.sub(' +', ' ', x))
df_DATATREE['DATATREE_ADDRESS'] = df_DATATREE['DATATREE_ADDRESS'].apply(lambda x: re.sub(' +', ' ', x))


- Create table with splitted addresses

In [ ]:
import usaddress
df1_MLS = df_MLS
def get_MLS_addr(df):
    global MLS_address
    try:
        MLS_address = usaddress.tag(df['MLS_ADDRESS'])
    except usaddress.RepeatedLabelError as e :
#         print('error')
        pass
    return MLS_address

df1_MLS['MLS_ADDRESS_Split'] = df1_MLS.apply(lambda df1_MLS : get_MLS_addr(df1_MLS), axis = 1)
test_MLS = df1_MLS.MLS_ADDRESS_Split.to_list()
values= []
for i in range(len(test_MLS)):
    values.append(test_MLS[i][0])
splited_MLS = pd.DataFrame(values)
df1_MLS.reset_index(inplace=True)
MLS_ADDRESS_final = pd.concat([df1_MLS,splited_MLS],axis=1)
del MLS_ADDRESS_final['index']
MLS_ADDRESS_final

In [ ]:
# deal with datatree
df1_DATATREE = df_DATATREE
def get_DATATREE_addr(df):
    global DT_ADDRESS
    try:
        DT_ADDRESS = usaddress.tag(df['DATATREE_ADDRESS'])
    except usaddress.RepeatedLabelError as e:
        pass
    return DT_ADDRESS
df1_DATATREE['DATATREE_ADDRESS_Split'] = df1_DATATREE.apply(lambda df1_DATATREE : get_DATATREE_addr(df1_DATATREE), axis = 1)
test_DATATREE = df1_DATATREE.DATATREE_ADDRESS_Split.to_list()
values= []
for i in range(len(test_DATATREE)):
    try:
        values.append(test_DATATREE[i][0])
    except:
        pass
splited_DATATREE = pd.DataFrame(values)
df1_DATATREE.reset_index(inplace=True)
DT_ADDRESS_final = pd.concat([df1_DATATREE,splited_DATATREE],axis=1)
del DT_ADDRESS_final['index']
DT_ADDRESS_final

- Address cleaning

In [ ]:
print(DT_ADDRESS_final.columns)
abbr_list = ['StreetNamePreDirectional','StreetNamePostType',
             'OccupancyType','StreetNamePostDirectional']
for col in abbr_list:
    print(f'{col}:\n'+ f'{DT_ADDRESS_final[col].unique()}') 

In [ ]:
print(MLS_ADDRESS_final.columns)
abbr_list = ['StreetNamePreDirectional','StreetNamePostType',
             'OccupancyType','StreetNamePostDirectional']
for col in abbr_list:
    print(f'{col}:\n'+ f'{MLS_ADDRESS_final[col].unique()}') 

In [ ]:
StreetNameDirectional_dict = {'west': 'w', 'south': 's', 'north': 'n','east': 'e'}
StreetNamePostType_dict = {'avenue': 'ave', 'highway': 'hwy', 'boulevard': 'blvd',
                          'street': 'st', 'parkway': 'pkwy', 'terrace': 'ter',
                          'lane': 'ln', 'place': 'pl', 'court': 'ct',
                          'plaza': 'plz', 'park': 'pk', 'square': 'sq', 'shore':'shr',
                          'branch': 'br', 'island': 'is', 'circle': 'cir', 'road':'rd',
                           'drive': 'dr', 'garden': 'gdn', 'grdn': 'gdn','trail': 'trl'} 

DT_ADDRESS_final.replace({"StreetNamePreDirectional": StreetNameDirectional_dict},inplace=True)
DT_ADDRESS_final.replace({"StreetNamePostType": StreetNamePostType_dict},inplace=True)

MLS_ADDRESS_final.replace({"StreetNamePreDirectional": StreetNameDirectional_dict},inplace=True)
MLS_ADDRESS_final.replace({"StreetNamePostType": StreetNamePostType_dict},inplace=True)

In [ ]:
print(MLS_ADDRESS_final['StreetNamePreDirectional'].unique())
print(DT_ADDRESS_final['StreetNamePreDirectional'].unique())
print(MLS_ADDRESS_final['StreetNamePostType'].unique())
print(DT_ADDRESS_final['StreetNamePostType'].unique())

In [ ]:
DT_ADDRESS_final[DT_ADDRESS_final['StreetNamePreDirectional'] == '62']

- Merge

In [ ]:
df_merge_node2 = pd.merge(DT_ADDRESS_final, MLS_ADDRESS_final,
                    how = 'left',
                    left_on = ['AddressNumber', 'StreetName','StreetNamePreDirectional','StreetNamePostType',
               'OccupancyIdentifier','OccupancyType', 'SITUSZIP5'],
                    right_on = ['AddressNumber', 'StreetName','StreetNamePreDirectional','StreetNamePostType',
               'OccupancyIdentifier','OccupancyType', 'POSTALCODE'])

In [ ]:
MLS_matched_address.extend(df_merge_node2[df_merge_node2['MLS_ADDRESS'].notnull()]['MLS_ADDRESS'].unique())
MLS_matched_address = list(set(MLS_matched_address))
len(MLS_matched_address)

In [ ]:
DATATREE_matched_address.extend(df_merge_node2[df_merge_node2['MLS_ADDRESS'].notnull()]['DATATREE_ADDRESS'].unique())
DATATREE_matched_address = list(set(DATATREE_matched_address))
len(DATATREE_matched_address)

In [ ]:
second_node_match = len(MLS_matched_address)
second_node_match_ratio = second_node_match/base
second_node_match_ratio

In [ ]:
MLS_unmatched_node2 = MLS_ADDRESS_final[MLS_ADDRESS_final['MLS_ADDRESS'].apply(lambda x: x not in MLS_matched_address)]
DATATREE_unmatched_node2 = DT_ADDRESS_final[DT_ADDRESS_final['DATATREE_ADDRESS'].apply(lambda x: x not in DATATREE_matched_address)]
MLS_unmatched_node2.reset_index(inplace=True)
DATATREE_unmatched_node2.reset_index(inplace=True)
del MLS_unmatched_node2['index']
del DATATREE_unmatched_node2['index']

# 4. Third Node - Match with names

### 4.1 Name cleaning

#### 4.1.1  remove punctuations

In [ ]:
from string import punctuation
import re
# punctuation -> space
def remove_punct(x):
    remove_punctiuation = str.maketrans(punctuation, ' '*len(punctuation))
    if x is not None:
        x = str(x)
        x = x.translate(remove_punctiuation)
        return x
    else:
        return x
MLS_unmatched_node2['NAME'] = MLS_unmatched_node2['NAME'].apply(lambda x: remove_punct(x))
DATATREE_unmatched_node2['OWNERNAME1FULL'] = DATATREE_unmatched_node2['OWNERNAME1FULL'].apply(lambda x: remove_punct(x))
# several spaces -> one space
def conver_space(x):
    if x is not None:
        x = str(x)
        x = re.sub(' +', ' ', x)
        return x.strip()
    else:
        return x
MLS_unmatched_node2['NAME'] = MLS_unmatched_node2['NAME'].apply(lambda x: conver_space(x))
DATATREE_unmatched_node2['OWNERNAME1FULL'] = DATATREE_unmatched_node2['OWNERNAME1FULL'].apply(lambda x: conver_space(x))


#### 4.1.2 Deal with corporation related names

- Replace the words related to corporations

In [ ]:
# Use the words to find possible replacement in sheets
# Incorpate - inc is kind of complicated, so no replacement
replace_name_dict_complicated = {'limited liability company': 'llc', 
                                 'company limited partnership': 'company ltd partnership', 
                                 'co limited partnership': 'co ltd partnership', 'company ltd': 'co ltd',
                                 'associates': 'assoc', 'limited liability partnerships': 'llp',
                                 'properties': 'prop', 'association': 'assn', 'estate': 'est',
                                 'condonminium': 'condo'}

MLS_unmatched_node2.replace({"NAME": replace_name_dict_complicated},inplace=True)
DATATREE_unmatched_node2.replace({"OWNERNAME1FULL": replace_name_dict_complicated},inplace=True)

In [ ]:
replace_name_dict_tail = {'incorporated': 'inc', 'corporation': 'corp', 'authority': 'a'}
def replace_corp(x, replace_dict):
    if x is None:
        return x
    import re
    for key, value in replace_dict.items():
        key = '(.*) ' + key + '$'
        if re.match(key, x):
            return re.search(key, x).group(1) + ' ' + value
    return x

In [ ]:
MLS_unmatched_node2['NAME'] = MLS_unmatched_node2['NAME'].apply(lambda x: replace_corp(x, replace_name_dict_tail))
DATATREE_unmatched_node2['OWNERNAME1FULL'] = DATATREE_unmatched_node2['OWNERNAME1FULL'].apply(lambda  x: replace_corp(str(x), replace_name_dict_tail))

### 4.2 Match table to using addresses whose owners only own one address in the DATATREE table

#### 4.2.1 Completely Match

- Find the addresses whose owners only own one address in the DATATREE table

In [ ]:
only_once = DATATREE_unmatched_node2[['DATATREE_ADDRESS','OWNERNAME1FULL']].groupby('OWNERNAME1FULL').count()
only_once = only_once[only_once['DATATREE_ADDRESS']==1]
remaining = only_once[only_once['DATATREE_ADDRESS']> 1]
once_list = list(only_once['DATATREE_ADDRESS'].index)
DATATREE_only_once = DATATREE_unmatched_node2[DATATREE_unmatched_node2['OWNERNAME1FULL'].isin(once_list)]

In [ ]:
df_merge_node3 = pd.merge(DATATREE_only_once, MLS_unmatched_node2, how = 'left', 
                            left_on = ['AddressNumber', 'StreetName', 'StreetNamePreDirectional', 'StreetNamePostType', 'SITUSZIP5'], 
                            right_on = ['AddressNumber', 'StreetName','StreetNamePreDirectional','StreetNamePostType', 'POSTALCODE'])

In [ ]:
df_merge_node3 = df_merge_node3[df_merge_node3['MLS_ADDRESS'].notnull()]

- only contains that one address has apt and the other does not have

In [ ]:
def is_null(x, y):
    if (x == 'nan') and (y == 'nan'):
        return True
    else:
        return False
df_merge_node3['Occupancy_x_none'] = df_merge_node3.apply(lambda x: is_null(str(x['OccupancyType_x']), str(x['OccupancyIdentifier_x'])), axis=1)
df_merge_node3['Occupancy_y_none'] = df_merge_node3.apply(lambda x: is_null(str(x['OccupancyType_y']), str(x['OccupancyIdentifier_y'])), axis=1)

In [ ]:
df_merge_node3 = df_merge_node3[df_merge_node3['Occupancy_y_none'] != df_merge_node3['Occupancy_x_none']]

In [ ]:
df_merge_node3_matched = df_merge_node3[df_merge_node3["NAME"] == df_merge_node3["OWNERNAME1FULL"]]

In [ ]:
MLS_matched_address.extend(list(df_merge_node3_matched['MLS_ADDRESS'].unique()))
MLS_matched_address = list(set(MLS_matched_address))
len(MLS_matched_address)

In [ ]:
DATATREE_matched_address.extend(list(df_merge_node3_matched['DATATREE_ADDRESS'].unique()))
DATATREE_matched_address = list(set(DATATREE_matched_address))
len(DATATREE_matched_address)

In [ ]:
third_node_match = len(MLS_matched_address)
base = len(df_DATATREE['DATATREE_ADDRESS'].unique())
third_node_match_ratio = third_node_match/base
third_node_match_ratio

#### 4.2.1 Deal with reversed order

- DATATREE

In [ ]:
df_merge_node3_not_matched = df_merge_node3[df_merge_node3["NAME"] != df_merge_node3["OWNERNAME1FULL"]]

In [ ]:
def is_null_order(x, y, z, name):
    if (x is None) and (y is None) and (z is None):
        return name
    else:
        return " ".join(sorted(name.split()))

In [ ]:
df_merge_node3_not_matched['OWNERNAME1FULL'] = df_merge_node3_not_matched.apply(lambda x: is_null_order(x['OWNER1LASTNAME'], x['OWNER1FIRSTNAME'], x['OWNER1MIDDLENAME'], x['OWNERNAME1FULL']), axis=1)

- MLS

- nltk pos_tag: does not work

In [ ]:
import nltk

In [ ]:
name = 'american tower corp'
name_list = list(name.split())
nltk.pos_tag(name_list)

In [ ]:
name = "norma martinez"
name_list = list(name.split())
nltk.pos_tag(name_list)

- spacy: return blank in some organizations and some human names

In [ ]:
#!pip install spacy
#!python3 -m spacy download en_core_web_sm
import spacy
from spacy import displacy
NER = spacy.load("en_core_web_sm")

In [ ]:
raw_text = 'american tower corp'
text1= NER(raw_text)
for word in text1.ents:
    print(word.text,word.label_)

In [ ]:
raw_text = '168 176 e 88 st corp'
text1= NER(raw_text)
for word in text1.ents:
    print(word.text,word.label_)

In [ ]:
raw_text = 'fine times inc'
text1= NER(raw_text)
for word in text1.ents:
    print(word.text,word.label_)

In [ ]:
raw_text = 'day spring baptist church'
text1= NER(raw_text)
for word in text1.ents:
    print(word.text,word.label_)

In [ ]:
raw_text = 'gerard mccarthy'
text1= NER(raw_text)
for word in text1.ents:
    print(word.text,word.label_)

In [ ]:
raw_text = 'norma martinez'
text1= NER(raw_text)
for word in text1.ents:
    print(word.text,word.label_)

In [ ]:
raw_text = 'martinez norma'
text1= NER(raw_text)
for word in text1.ents:
    print(word.text,word.label_)

- CoreNLPParser(https://github.com/nltk/nltk/wiki/Stanford-CoreNLP-API-in-NLTK): cannot identify reversed order names 

In [ ]:
from nltk.parse import CoreNLPParser

In [ ]:
ner_tagger = CoreNLPParser(url='http://localhost:9000', tagtype='ner')

In [ ]:
list(ner_tagger.tag(('mccarthy gerard'.split())))

In [ ]:
list(ner_tagger.tag(('gerard mccarthy'.split())))

In [ ]:
def covert_human_name(name):
    if name is None:
        return name
    else:
        text =  NER(name)
        labels = []
        for word in text.ents:
            labels.append(word.label_)
        if 'PERSON' in labels:
            return " ".join(sorted(name.split()))
        else: 
            return name

In [ ]:
def covert_human_name_col(col):
    import spacy
    from spacy import displacy
    NER = spacy.load("en_core_web_sm")
    col = col.apply(lambda x: covert_human_name(x))
    return col

In [ ]:
df_merge_node3_not_matched['NAME'] = covert_human_name_col(df_merge_node3_not_matched['NAME'])

In [ ]:
df_merge_node3_not_matched[df_merge_node3_not_matched["NAME"] == df_merge_node3_not_matched["OWNERNAME1FULL"]]

- Reverse could help us find only 2 more matching addresses